# Crime Data Analysis

Crime statistics are crucial in helping law enforcement and public safety agencies better understand crime patterns and develop preventive measures. This project aims to analyze crime data to uncover trends, identify hotspots, and provide actionable insights that can aid in reducing crime rates.

In [ ]:
# Install Folium package to visualize maps
pip install folium

In [ ]:
# Import Pandas and Pymysql modules
import pandas as pd
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [ ]:
# Load the crime_data.csv to Pandas DataFrame
df = pd.read_csv(r"C:\Users\Hari\OneDrive - Wiley\Desktop\MyDesktop_14102024\Desktop\Data Science Bootcamp\Mini_Capstone_Project\Dataset\crime_data.csv")


In [ ]:
# Replace the NaN values with None
df = df.where(df.notnull(), None)

In [ ]:
# Establish a database connection
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Virat200599#',
                             database='crime_data_db')

In [ ]:
# Create a cursor object
cursor = connection.cursor()

# Create a crime data table

In [ ]:
# Create crime_data Table in crime_data_db database
query = '''create table crime_data(
          dr_no int,
          date_reported DATE,
          date_occurred DATE,
          area_name varchar(100),
          crime_code int,
          crime_code_description varchar(255),
          victim_age int,
          victim_sex varchar(10),
          premise_description varchar(255),
          status varchar(100),
          location varchar(100),
          lat double,
          lon double);'''

cursor.execute(query)

# Insert values to the crime_data table using pymysql
for index, row in df.iterrows():
    query = r'''insert into crime_data(dr_no, date_reported, date_occurred, area_name, crime_code, crime_code_description,
    victim_age, victim_sex, premise_description, status, location, lat, lon)
    values(%s,str_to_date(%s, '%%d-%%m-%%Y'),str_to_date(%s, '%%d-%%m-%%Y'),%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
    cursor.execute(query, tuple(row))

# Save changes in Database
connection.commit()

# Crime Data Exploratory Analysis

In [ ]:
# Query to fetch total record count
cursor.execute('select count(*) from crime_data;')
total_records = cursor.fetchone()[0]

# Query to fetch unique crime_code records
cursor.execute('select count(distinct crime_code) from crime_data;')
unique_crime_codes_count = cursor.fetchone()[0]

# Print the total_records and unique_crime_codes_count
print(f'Total records: {total_records}')
print(f'Unique Crime codes: {unique_crime_codes_count}')

In [ ]:
# Query to fetch unique crime_code and their descriptions
cursor.execute('select distinct crime_code, crime_code_description from crime_data order by crime_code;')
unique_crime_codes_descriptions = cursor.fetchall()

# Print the unique crime_codes and their descriptions
print('Unique crime codes and their descriptions')
for crime_code, description in unique_crime_codes_descriptions:
    print(crime_code, description)

# Temporal Analysis

In [ ]:
# Convert the string data to date with specified format
df['DATE_OCC'] = pd.to_datetime(df['DATE_OCC'], format='%d-%m-%Y')

# Group the data on Month of the Crime_occurred_date to analyze trends over time
res = df.groupby(df['DATE_OCC'].dt.to_period('M')).size()

# Plot Line graph to analyze the trend over time
res.plot(kind='line', marker='o', label='Monthly values')
plt.title('Crime Trends Over Time')
plt.xlabel('Period in Month')
plt.ylabel('Number of Crimes')
plt.legend(labelcolor='red')
plt.show()

# Spatial Analysis

In [ ]:
# Create a map centered around an average location
crime_map = folium.Map(location=[df['LAT'].mean(), df['LON'].mean()], zoom_start=12)

# Add points to the map
for _, row in df.iterrows():
    folium.CircleMarker([row['LAT'], row['LON']],
                        radius=2,
                        color='red').add_to(crime_map)

In [ ]:
# Display the Crime Hotspot map
crime_map

# Victim Demographics Analysis

In [ ]:
# Distribution of Victim Ages

sns.histplot(x=df['Vict_Age'].dropna(), bins=25)
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Distribution of Victim Ages')
plt.show()

In [ ]:
# Group the data on Victim Sex 

grouped_data = df.groupby(by='Vict_Sex').size().sort_values(ascending=False)
grouped_data.index = ['Male', 'Female', 'Others']

In [ ]:
# Crime Rates by  Victim Gender

sns.countplot(x=df['Vict_Sex'], palette='Set1')
plt.title('Crime Rates by  Victim')
plt.xlabel('Victim Sex')
plt.ylabel('Number of Victims')
plt.show()

In [ ]:
# Difference in crime rates between Various genders

plt.pie(x=grouped_data, autopct='%1.1f%%', labels=grouped_data.index)
plt.title('Difference in crime rates between Various genders')
plt.show()

In [ ]:
# Location with most crimes reported
location_counts = df.groupby(by='Location').size().sort_values(ascending=False).head(1)
print(location_counts)

In [ ]:
# Crimes based on Premise location

grouped_data = df.groupby(by='Premis_Desc').size().sort_values(ascending=False)
grouped_data = grouped_data[grouped_data>1].head(5)

# Common premises descriptions where crimes occur

plt.pie(x=grouped_data, autopct='%1.1f%%', labels=grouped_data.index)
plt.title('Common premises descriptions where crimes occur')
plt.show()

# Status Analysis

In [ ]:
# Status of Reported Crimes
status_counts = df.groupby(by='Status').size().sort_values(ascending=False)
sns.barplot(x=status_counts.index, y=status_counts.values, palette='deep')
plt.title('Status of Reported Crimes')
plt.xlabel('Status')
plt.ylabel('Count')
plt.show()

# Crime Code Analysis

In [ ]:
# Distribution of Crime Codes
sns.kdeplot(x=df['Crm_Cd'].dropna(), fill=True, color='red')
plt.xlabel('Crime codes')
plt.ylabel('Probability of occurrence')
plt.title('Distribution of Crime Codes')
plt.show()